## Example still work in progress

In [1]:
import datetime, time
import simpy

import shapely.geometry


import pandas as pd
import openclsim.core as core
import openclsim.model as model

import datetime

import numpy as np

In [2]:
simulation_start = datetime.datetime.now()

my_env = simpy.Environment(initial_time=time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

registry = {}
keep_resources = {}

In [3]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)


TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

In [4]:
location_from_site = shapely.geometry.Point(5.1, 52)
location_to_site = shapely.geometry.Point(5, 52.1)
location_to_site_2 = shapely.geometry.Point(5, 52.2)

data_from_site = {
    "env": my_env,
    "name": "Winlocatie",
    "geometry": location_from_site,
    "capacity": 100,
    "level": 100,
}

data_to_site = {
    "env": my_env,
    "name": "Dumplocatie",
    "geometry": location_to_site,
    "capacity": 55,
    "level": 0,
}


from_site = Site(**data_from_site)
to_site = Site(**data_to_site)


In [5]:
data_cutter = {
    "env": my_env,
    "name": "Cutter_1",
    "geometry": location_from_site,
    "capacity": 5,
    "compute_v": lambda x: 10,
    "loading_rate": 3600/5,
    "unloading_rate": 3600/5
}


data_barge_1 = {
    "env": my_env,
    "name": "Barge_1",
    "geometry": location_to_site,
    "capacity": 5,
    "compute_v": lambda x: 10,
    "loading_rate": 3600/5,
    "unloading_rate": 3600/5
}


data_barge_2 = {
    "env": my_env,
    "name": "Barge_2",
    "geometry": location_to_site,
    "capacity": 5,
    "compute_v": lambda x: 10,
    "loading_rate": 3600/5,
    "unloading_rate": 3600/5
}

cutter = TransportProcessingResource(**data_cutter)
barge_1 = TransportProcessingResource(**data_barge_1)
barge_2 = TransportProcessingResource(**data_barge_2)

In [6]:
requested_resources = {}

single_run = [
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing empty_1",
        "registry": registry,
        "mover": barge_1,
        "destination": from_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer MP_1",
        "registry": registry,
        "processor": cutter,
        "origin": from_site,
        "destination": barge_1,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
#         "requested_resources":requested_resources
    }),
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing filled_1",
        "registry": registry,
        "mover": barge_1,
        "destination": to_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer TP_1",
        "registry": registry,
        "processor": barge_1,
        "origin": barge_1,
        "destination": to_site,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
#         "requested_resources":requested_resources
    })
]


sequential_activity_data = {
    "env": my_env,
    "name": "Single run process_1",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data)

while_activity = model.WhileActivity(**{
    "env": my_env,
    "name": "while_1",
    "registry": registry,
    "sub_processes": [activity],
    "condition_event": [{"type":"container", "concept": to_site, "state":"full"}],
    "postpone_start": False,
})

In [7]:
single_run = [
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing empty_2",
        "registry": registry,
        "mover": barge_2,
        "destination": from_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer MP_2",
        "registry": registry,
        "processor": cutter,
        "origin": from_site,
        "destination": barge_2,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
    }),
    model.MoveActivity(**{
        "env": my_env,
        "name": "sailing filled_2",
        "registry": registry,
        "mover": barge_2,
        "destination": to_site,
        "postpone_start": True,
    }),
    model.ShiftAmountActivity(**{
        "env": my_env,
        "name": "Transfer TP_2",
        "registry": registry,
        "processor": barge_2,
        "origin": barge_2,
        "destination": to_site,
        "amount": 5,
        "duration": 3600,
        "postpone_start": True,
    })
]


activity = model.SequentialActivity(** {
    "env": my_env,
    "name": "Single run process_2",
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
#     "keep_resources"
})

while_activity = model.WhileActivity(**{
    "name": "while_2",
    "env": my_env,
    "registry": registry,
    "sub_processes": [activity],
    "condition_event": [{"type":"container", "concept": to_site, "state":"full"}],
    "postpone_start": False,
})

In [8]:
my_env.run()

In [9]:
# !! BUGG !!
#  why we need reservations:
# !! BUGG !!

print(f"Level of the to_site: {to_site.container.get_level()}, Capacity of the to_site {to_site.container.get_capacity()}")

Level of the to_site: 60.0, Capacity of the to_site 55
